In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
%matplotlib inline

In [2]:
df = pd.read_csv('/home/tomek/ib_tools/blotter/backtest_2019-11-08_22-39.csv')
df.time = df.time.astype('datetime64')
df.set_index('time', inplace=True)
df

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2018-02-09 09:28:00,2019-11-08 22:41:04.129398,GCZ9,SELL,1,1316.150,[1],1,entry,1,1.3,USD,0.0,2019-11-08 22:41:04.129520
2018-02-09 13:31:00,2019-11-08 22:41:04.496455,GCZ9,BUY,1,1318.880,[2],2,stop-out,2,1.3,USD,-273.0,2019-11-08 22:41:04.496506
2018-02-09 13:45:00,2019-11-08 22:41:04.547793,YM DEC 19,SELL,1,23822.100,[3],3,entry,3,1.3,USD,0.0,2019-11-08 22:41:04.547921
2018-02-09 14:10:00,2019-11-08 22:41:04.626944,CLZ9,SELL,1,60.354,[4],5,entry,4,1.3,USD,0.0,2019-11-08 22:41:04.627054
2018-02-09 14:39:00,2019-11-08 22:41:04.778000,YM DEC 19,BUY,1,23991.700,[5],4,stop-out,5,1.3,USD,-848.0,2019-11-08 22:41:04.778050
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-01 14:01:00,2019-11-09 00:40:02.804454,CLZ9,BUY,1,54.849,[4406],5081,entry,4406,1.3,USD,0.0,2019-11-09 00:40:02.805350
2019-11-01 14:20:00,2019-11-09 00:40:03.453655,NQZ9,BUY,1,8127.975,[4407],5083,entry,4407,1.3,USD,0.0,2019-11-09 00:40:03.454567
2019-11-01 14:36:00,2019-11-09 00:40:04.127382,YM DEC 19,BUY,1,27132.100,[4408],5085,entry,4408,1.3,USD,0.0,2019-11-09 00:40:04.128257


In [3]:
df[df.reason == 'entry'].price.count()

2204

In [4]:
df[df.reason != 'entry'].price.count()

2206

In [5]:
df.price.count()

4410

In [7]:
df.realizedPNL.sum()

87728.25

In [16]:
# percentage of winning transactions
pct_win = round(df[df.realizedPNL>0].realizedPNL.count() / df[df.realizedPNL != 0].realizedPNL.count(), 2)
pct_win

0.25

In [14]:
# avg win, avg lose, ratio
avg_win = round(df[df.realizedPNL > 0].realizedPNL.mean())
avg_loss = round(df[df.realizedPNL < 0].realizedPNL.mean())
ratio = round(abs(avg_win / avg_lose), 2)
avg_win, avg_loss, ratio

(1282.0, -374.0, 3.43)

In [10]:
df.realizedPNL.min(), df.realizedPNL.max(), round(abs(df.realizedPNL.max()/df.realizedPNL.min()))

(-1696.0, 6320.0, 4.0)

In [20]:
# EV
ev = (pct_win * avg_win) + ((1 - pct_win) * avg_loss)
ev

40.0

In [21]:
ev * df.price.count()/2

88200.0

In [13]:
df.realizedPNL.resample('M').sum()

time
2018-02-28    -4205.46
2018-03-31    21229.45
2018-04-30    -1669.50
2018-05-31    -5163.73
2018-06-30     7281.57
2018-07-31    -4152.25
2018-08-31     9352.29
2018-09-30   -11092.59
2018-10-31    -5868.89
2018-11-30     8907.55
2018-12-31    37219.75
2019-01-31    -8759.74
2019-02-28     6968.17
2019-03-31     5170.19
2019-04-30     9282.82
2019-05-31   -12997.88
2019-06-30    32269.30
2019-07-31    -4830.47
2019-08-31    13219.17
2019-09-30    -3033.75
2019-10-31    -1392.25
2019-11-30       -5.50
Freq: M, Name: realizedPNL, dtype: float64